# Libraries and Frameworks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#optional installation for torchsummary
!pip install torchsummary

In [ ]:
#standard libraries
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from matplotlib.colors import ListedColormap, BoundaryNorm
import pandas as pd
from datetime import datetime
import random
import time
import copy

#augmentation
from albumentations.pytorch import ToTensorV2
import albumentations as A

#torch
import torch
from torch.utils.data import Dataset, SubsetRandomSampler, DataLoader, random_split
from torch.cuda.amp import GradScaler
#from torchvision.transforms import v2
import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR, MultiStepLR, ReduceLROnPlateau, ExponentialLR, CosineAnnealingLR
from torchsummary import summary

#import onnx
import torch.onnx

print(f'GPU on: {torch.cuda.is_available()}')
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
_today=datetime.today().strftime('%Y-%m-%d')

In [ ]:
#clear gpu cuda cache
torch.cuda.empty_cache()

In [ ]:
#clone repo
!git clone https://github.com/t0wgster/thesis_constants.git
!cd thesis_constants && git pull

#load in important functions
from thesis_constants.functions_and_constants import *
from thesis_constants.functions_and_constants import _WHDataset_10_classes, _WH_HSI_Dataset, _WH_RGB_HSI_Dataset
from thesis_constants.visualisation_and_evaluation import *
from thesis_constants.models import *
from thesis_constants.post_processing import *


In [ ]:
#optional
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
#make training deterministic, important for comparing different models
seed_everything(81)
VISUALIZE = True

# Get the current date and time
current_date = datetime.now()

# Format the current date as "dd-mm-yyyy"
CURRENT_DATE = current_date.strftime("%d-%m-%Y")

print("Current date (dd-mm-yyyy format):", CURRENT_DATE)



Current date (dd-mm-yyyy format): 11-06-2024


# Initiate Dataset and Dataloaders for Training/Evaluation

In [ ]:
rgb_dir_train = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/train/rgb'
hsi_dir_train = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/train/hsi'
mask_dir_train = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/train/masks'

rgb_dir_test = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/test/rgb'
hsi_dir_test = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/test/hsi'
mask_dir_test = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/test/masks'

rgb_dir_val = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/val/rgb'
hsi_dir_val = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/val/hsi'
mask_dir_val = '/content/drive/MyDrive/Master/HSI/train-test-split-1106/split_resized_dataset_rgb_mask_hsi/val/masks'

In [ ]:
VAL_BATCH_SIZE=12
TRAIN_BATCH_SIZE=12
NUM_WORKERS=2

train_dataset = _WH_RGB_HSI_Dataset(rgb_dir_train, hsi_dir_train, mask_dir_train,
                                    transform = sf_transformation)
test_dataset = _WH_RGB_HSI_Dataset(rgb_dir_test, hsi_dir_test, mask_dir_test,
                                   transform = sf_transformation)
val_dataset = _WH_RGB_HSI_Dataset(rgb_dir_val, hsi_dir_val, mask_dir_val,
                                  transform = sf_transformation)

train_dataset_final = _WH_RGB_HSI_Dataset(rgb_dir_train, hsi_dir_train, mask_dir_train,
                                          transform = sf_no_transformation)
test_dataset_final = _WH_RGB_HSI_Dataset(rgb_dir_test, hsi_dir_test, mask_dir_test,
                                         transform = sf_no_transformation)
val_dataset_final = _WH_RGB_HSI_Dataset(rgb_dir_val, hsi_dir_val, mask_dir_val,
                                        transform = sf_no_transformation)

generator1 = torch.Generator().manual_seed(42)

print(f'Train Dataset Length: {len(train_dataset)}')
print(f'Test Dataset Length: {len(test_dataset)}')
print(f'Validation Dataset Length: {len(val_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE,
                          shuffle=True, num_workers=2)
train_loader_final = DataLoader(train_dataset_final, batch_size=TRAIN_BATCH_SIZE,
                                shuffle=True, num_workers=2)

val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE,
                        shuffle=True, drop_last=True, num_workers=2)
val_loader_final = DataLoader(val_dataset_final, batch_size=VAL_BATCH_SIZE,
                              shuffle=True, drop_last=True, num_workers=2)

test_loader = DataLoader(test_dataset, batch_size=2,
                         shuffle=False, num_workers=2)
test_loader_final = DataLoader(test_dataset_final, batch_size=2,
                               shuffle=False, num_workers=2)

NameError: name '_WH_RGB_HSI_Dataset' is not defined

# Check Augmentation Results

In [ ]:
print('Legend:')
for i, color in enumerate(COLORS_LONG):
      print(f'{TXT_COLORS_LONG[i]} -> {CLASSES_LONG[i]}')
print('\033[0m - - - - -')

for i in range(0, 20, 2):
    print(i)
    image, hsi_image, mask = train_dataset[i]
    image2, hsi_image2, mask2 = train_dataset[i+1]

    image = image.numpy().transpose((1, 2, 0))
    hsi_image = hsi_image.numpy().transpose((1, 2, 0))[:,:,0]

    image2 = image2.numpy().transpose((1, 2, 0))
    hsi_image2 = hsi_image2.numpy().transpose((1, 2, 0))[:,:,0]

    #img_arr = np.asarray(image.permute(1,2,0))
    mask_arr = np.asarray(mask)

    #img_arr2 = np.asarray(image2.permute(1,2,0))
    mask_arr2 = np.asarray(mask2)

    fig, axs = plt.subplots(1,6, figsize=(16,16))
    axs[0].imshow(image)
    axs[1].imshow(hsi_image)
    axs[2].imshow(mask_arr, cmap=cmap_long, norm=norm_long)

    axs[3].imshow(image2)
    axs[4].imshow(hsi_image2)
    axs[5].imshow(mask_arr2, cmap=cmap_long, norm=norm_long)

    for ax in axs:
        ax.axis('off')

    plt.show()

# Training Hyperparameters

In [ ]:
LEARNING_RATE = 0.0003
NUM_EPOCHS = 300
NUM_EPOCHS_FINAL = 3
PATIENCE = 100

WEIGHTS = torch.tensor([1.0 ,1.0, 3.0 ,10.0
                        ,25.0 ,10.0 ,10.0
                        ,12.0 ,1.0 ,10.0]).to(DEVICE)

NameError: name 'torch' is not defined

# Data Level

# Training Parameters - Data Level Fusion Model

In [ ]:
sf_model = unet_model_gelu_data_level_fusion(in_channels_hsi=6, out_channels=10).to(DEVICE)


ce_loss_fn = nn.CrossEntropyLoss(weight=WEIGHTS)
dice_loss_fn=DiceLoss(n_classes=10)

optimizer = Adam(sf_model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
scaler = torch.cuda.amp.GradScaler()

scheduler = ExponentialLR(optimizer, last_epoch=-1, gamma=0.9)
source = 'sf'

In [ ]:
summary(sf_model, input_size=[(3,384,320), (6,384,320)])

# Model Training Data Level

In [ ]:
avg_train_loss_list=[]
avg_val_loss_list=[]
avg_train_iou_list=[]
avg_val_iou_list=[]
softmax = nn.Softmax(dim=1)
data_level_name = 'DataLevel'

In [ ]:
trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(sf_model, train_loader, val_loader_final, NUM_EPOCHS,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=PATIENCE, model_name=data_level_name,
                                                                            data_source = source)

In [ ]:
trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(trained_model, train_loader_final, val_loader_final, NUM_EPOCHS_FINAL,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=0, model_name=data_level_name,
                                                                            data_source = source)

In [ ]:
#plot_range = range(NUM_EPOCHS+NUM_EPOCHS_FINAL)

fig, ax = plt.subplots(figsize=(9,6))
ax.plot(range(len(avg_train_loss_list)), avg_train_loss_list, marker='o', linestyle='-', label='Training Loss', color='blue')

# Create a twin Axes sharing the xaxis
ax2 = ax.twinx()
ax2.plot(range(len(avg_val_loss_list)), avg_val_loss_list, marker='o', linestyle='-', label='Validation Loss', color='orange')

# Set labels and title
ax.set_xlabel('Epochs')
ax.set_ylabel('Training Loss', color='blue')
ax2.set_ylabel('Validation Loss', color='orange')
ax.set_title('Training vs Validation Loss')

# Show legend for both axes
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.grid(True)
plt.show()

# Evaluation - Data Level Fusion Model - Final Model

In [ ]:
test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(trained_model, test_dataset_final, data_source='sf', visualize=VISUALIZE, mask_shape = (384, 320))

In [ ]:
calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=data_level_name)

# Evaluation - Data Level Fusion Model - Best Model

In [ ]:

path = '/kaggle/working/'
model_name = f'best_model_{_today}_{data_level_name}.pt'
model_path = os.path.join(path, model_name)
best_model_dl = load_model(unet_model_gelu_data_level_fusion(in_channels_hsi=6, out_channels=10), optimizer, scaler, model_path)

test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(best_model_dl, test_dataset_final, data_source='sf', visualize=VISUALIZE, mask_shape = (384, 320))

calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=data_level_name)

calculate_model_inference_time(best_model_dl, val_loader_final, 'sf')

# Feature Level

# Training Parameters - Feature Level Fusion Model

In [ ]:
sf_model = unet_model_gelu_feature_level_fusion(in_channels_hsi=6, out_channels=10).to(DEVICE)

ce_loss_fn = nn.CrossEntropyLoss(weight=WEIGHTS)
dice_loss_fn=DiceLoss(n_classes=10)

optimizer = Adam(sf_model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
scaler = torch.cuda.amp.GradScaler()

scheduler = ExponentialLR(optimizer, last_epoch=-1, gamma=0.9)
source = 'sf'

In [ ]:
summary(sf_model, input_size=[(3,384,320), (6,384,320)])

# Model Training Feature Level

In [ ]:
avg_train_loss_list=[]
avg_val_loss_list=[]
avg_train_iou_list=[]
avg_val_iou_list=[]
softmax = nn.Softmax(dim=1)
feature_level_name = 'FeatureLevel'

trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(sf_model, train_loader, val_loader_final, NUM_EPOCHS,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=PATIENCE, model_name=feature_level_name,
                                                                            data_source = source)

In [ ]:
trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(trained_model, train_loader_final, val_loader_final, NUM_EPOCHS_FINAL,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=0, model_name=feature_level_name,
                                                                            data_source = source)

In [ ]:
#plot_range = range(NUM_EPOCHS+NUM_EPOCHS_FINAL)

fig, ax = plt.subplots(figsize=(9,6))
ax.plot(range(len(avg_train_loss_list)), avg_train_loss_list, marker='o', linestyle='-', label='Training Loss', color='blue')

# Create a twin Axes sharing the xaxis
ax2 = ax.twinx()
ax2.plot(range(len(avg_val_loss_list)), avg_val_loss_list, marker='o', linestyle='-', label='Validation Loss', color='orange')

# Set labels and title
ax.set_xlabel('Epochs')
ax.set_ylabel('Training Loss', color='blue')
ax2.set_ylabel('Validation Loss', color='orange')
ax.set_title('Training vs Validation Loss')

# Show legend for both axes
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.grid(True)
plt.show()

# Evaluation - Feature Level Fusion Model - Final Model

In [ ]:
test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(trained_model, test_dataset_final, data_source='sf', visualize=VISUALIZE, mask_shape = (384, 320))

In [ ]:
calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=feature_level_name)

# Evaluation - Feature Level Fusion Model - Best Model

In [ ]:

path = '/kaggle/working/'
model_name = f'best_model_{_today}_{feature_level_name}.pt'
model_path = os.path.join(path, model_name)
best_model_fl = load_model(unet_model_gelu_feature_level_fusion(in_channels_hsi=6, out_channels=10), optimizer, scaler, model_path)

test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(best_model_fl, test_dataset_final, data_source='sf', visualize=VISUALIZE, mask_shape = (384, 320))

calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=data_level_name)

calculate_model_inference_time(best_model_fl, val_loader_final, 'sf')

# RGB - Unet Classic/Baseline

# Training Parameters - RGB Unet Classic Model

In [ ]:
classic_model = unet_model_classic(out_channels=10).to(DEVICE)

ce_loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(classic_model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
scaler = torch.cuda.amp.GradScaler()

scheduler = ExponentialLR(optimizer, last_epoch=-1, gamma=0.9)
source = 'rgb'

In [ ]:
summary(classic_model, (3, 384, 320))

# Model Training RGB Unet Classic

In [ ]:
avg_train_loss_list=[]
avg_val_loss_list=[]
avg_train_iou_list=[]
avg_val_iou_list=[]
softmax = nn.Softmax(dim=1)
classic_name = 'Classic'

trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training(classic_model, train_loader, val_loader_final, NUM_EPOCHS,
                                                                            ce_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=PATIENCE, model_name=classic_name,
                                                                            data_source = source)




In [ ]:
trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training(trained_model, train_loader_final, val_loader_final, NUM_EPOCHS_FINAL,
                                                                            ce_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=0, model_name=classic_name,
                                                                            data_source = source)

In [ ]:
#plot final training and validation loss
fig, ax = plt.subplots(figsize=(9,6))
ax.plot(range(len(avg_train_loss_list)), avg_train_loss_list, marker='o', linestyle='-', label='Training Loss', color='blue')

# Create a twin Axes sharing the xaxis
ax2 = ax.twinx()
ax2.plot(range(len(avg_val_loss_list)), avg_val_loss_list, marker='o', linestyle='-', label='Validation Loss', color='orange')

# Set labels and title
ax.set_xlabel('Epochs')
ax.set_ylabel('Training Loss', color='blue')
ax2.set_ylabel('Validation Loss', color='orange')
ax.set_title('Training vs Validation Loss')

# Show legend for both axes
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.grid(True)
plt.show()

# Evaluation - RGB Unet Classic Model - Final Model

In [ ]:
# evaluate model and visualize model outcome, optionally project confusion matrix
test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(trained_model, test_dataset_final, data_source='rgb', visualize=VISUALIZE, mask_shape = (384, 320))

In [ ]:
# calculate average model metrics
calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=classic_name)

# Evaluation - RGB Unet Classic Model - Best Model

In [ ]:
# save model and evaluate best model
path = '/kaggle/working/'
model_name = f'best_model_{_today}_{classic_name}.pt'
model_path = os.path.join(path, model_name)
best_model_unet = load_model(unet_model_classic(out_channels=10), optimizer, scaler, model_path)

test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(best_model_unet, test_dataset_final, data_source='rgb', visualize=VISUALIZE, mask_shape = (384, 320))

calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=classic_name)

calculate_model_inference_time(best_model_unet, val_loader_final, 'rgb')

# RGB - GELU

# Training Parameters - RGB GELU Model

In [ ]:
# initiate GEU model with 10 output classes and project model to CUDA
gelu_model = unet_model_gelu(out_channels=10).to(DEVICE)

# define CE and Dice loss
ce_loss_fn = nn.CrossEntropyLoss(weight=WEIGHTS)
dice_loss_fn=DiceLoss(n_classes=10)

# define Adam optimizer and learning rate
optimizer = Adam(gelu_model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
scaler = torch.cuda.amp.GradScaler()

# optional scheduler
scheduler = ExponentialLR(optimizer, last_epoch=-1, gamma=0.9)

In [ ]:
#show a model summary
summary(gelu_model, (3, 384, 320))

# Training RGB GELU Model

In [ ]:
#lists to store train and validation loss, for plotting purposes
avg_train_loss_list=[]
avg_val_loss_list=[]
avg_train_iou_list=[]
avg_val_iou_list=[]

gelu_name = 'Gelu'

# Training for defined number of epochs
model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(gelu_model, train_loader, val_loader_final, NUM_EPOCHS,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=PATIENCE, model_name=gelu_name,
                                                                            data_source = source)

# Training for additional 3 epochs with no augmentations
model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(model, train_loader_final, val_loader_final, NUM_EPOCHS_FINAL,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=0, model_name=gelu_name,
                                                                            data_source = source)

NameError: name 'nn' is not defined

In [ ]:
# Training for additional 3 epochs with no augmentations
trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(trained_model, train_loader_final, val_loader_final, NUM_EPOCHS_FINAL,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=0, model_name=gelu_name,
                                                                            data_source = source)

NameError: name 'sf_model_training_multiloss' is not defined

In [ ]:
#plot_range = range(NUM_EPOCHS+NUM_EPOCHS_FINAL)

fig, ax = plt.subplots(figsize=(9,6))
ax.plot(range(len(avg_train_loss_list)), avg_train_loss_list, marker='o', linestyle='-', label='Training Loss', color='blue')

# Create a twin Axes sharing the xaxis
ax2 = ax.twinx()
ax2.plot(range(len(avg_val_loss_list)), avg_val_loss_list, marker='o', linestyle='-', label='Validation Loss', color='orange')

# Set labels and title
ax.set_xlabel('Epochs')
ax.set_ylabel('Training Loss', color='blue')
ax2.set_ylabel('Validation Loss', color='orange')
ax.set_title('Training vs Validation Loss')

# Show legend for both axes
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.grid(True)
plt.show()

# Evaluation - RGB GELU Model - Final Model

In [ ]:
test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(trained_model,
                                                                                                                                                                                 test_dataset_final,
                                                                                                                                                                                 data_source='rgb',
                                                                                                                                                                                 visualize = VISUALIZE,
                                                                                                                                                                                 mask_shape = (384, 320))

In [ ]:
calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=gelu_name)

# Evaluation - RGB GELU Model - Best Model

In [ ]:

path = '/kaggle/working/'
model_name = f'best_model_{_today}_{gelu_name}.pt'
model_path = os.path.join(path, model_name)
best_model_gelu = load_model(unet_model_gelu(out_channels=10), optimizer, scaler, model_path)

test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(best_model_gelu, test_dataset_final, data_source='rgb', visualize=VISUALIZE, mask_shape = (384, 320))

calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=gelu_name)


calculate_model_inference_time(best_model_gelu, val_loader_final, 'rgb')

# RGB - ResNet

# Training Parameters - RGB ResNet Model

In [ ]:
resnet_model = UNetWithResnet50Encoder(n_classes=10).to(DEVICE)

ce_loss_fn = nn.CrossEntropyLoss(weight=WEIGHTS)
dice_loss_fn=DiceLoss(n_classes=10)

optimizer = Adam(resnet_model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
scaler = torch.cuda.amp.GradScaler()

scheduler = ExponentialLR(optimizer, last_epoch=-1, gamma=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
summary(resnet_model, (3, 384, 320))

# Training RGB ResNet Model

In [ ]:
avg_train_loss_list=[]
avg_val_loss_list=[]
avg_train_iou_list=[]
avg_val_iou_list=[]
softmax = nn.Softmax(dim=1)
resnet_name = 'ResNet'

trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(resnet_model, train_loader, val_loader_final, NUM_EPOCHS,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=PATIENCE, model_name=resnet_name,
                                                                            data_source = source)

In [ ]:
trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(trained_model, train_loader_final, val_loader_final, NUM_EPOCHS_FINAL,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=0, model_name=resnet_name,
                                                                            data_source = source)

In [ ]:
#plot_range = range(NUM_EPOCHS+NUM_EPOCHS_FINAL)

fig, ax = plt.subplots(figsize=(9,6))
ax.plot(range(len(avg_train_loss_list)), avg_train_loss_list, marker='o', linestyle='-', label='Training Loss', color='blue')

# Create a twin Axes sharing the xaxis
ax2 = ax.twinx()
ax2.plot(range(len(avg_val_loss_list)), avg_val_loss_list, marker='o', linestyle='-', label='Validation Loss', color='orange')

# Set labels and title
ax.set_xlabel('Epochs')
ax.set_ylabel('Training Loss', color='blue')
ax2.set_ylabel('Validation Loss', color='orange')
ax.set_title('Training vs Validation Loss')

# Show legend for both axes
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.grid(True)
plt.show()

# Evaluation - RGB ResNet Model - Final Model

In [ ]:
test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(trained_model, test_dataset_final, data_source='rgb', visualize = VISUALIZE, mask_shape = (384, 320))

In [ ]:
calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=resnet_name)

# Evaluation - RGB ResNet Model - Best Model

In [ ]:

path = '/kaggle/working/'
model_name = f'best_model_{_today}_{resnet_name}.pt'
model_path = os.path.join(path, model_name)
best_model_resnet = load_model(UNetWithResnet50Encoder(n_classes=10), optimizer, scaler, model_path)

test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(best_model_resnet, test_dataset_final, data_source='rgb', visualize=VISUALIZE, mask_shape = (384, 320))

calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=resnet_name)


calculate_model_inference_time(best_model_resnet, val_loader_final, 'rgb')

# HSI - GELU PCA

# Training Parameters - HSI GELU Model

In [ ]:
hsi_unet_pca_model = hsi_unet_model_gelu_pca(6).to(DEVICE)

ce_loss_fn = nn.CrossEntropyLoss(weight=WEIGHTS)
dice_loss_fn=DiceLoss(n_classes=10)

optimizer = Adam(hsi_unet_pca_model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
scaler = torch.cuda.amp.GradScaler()

scheduler = ExponentialLR(optimizer, last_epoch=-1, gamma=0.9)
source='hsi'

In [ ]:
summary(hsi_unet_pca_model, (6, 384, 320))

# Training HSI GELU Model

In [ ]:
avg_train_loss_list=[]
avg_val_loss_list=[]
avg_train_iou_list=[]
avg_val_iou_list=[]
softmax = nn.Softmax(dim=1)
hsi_unet_pca_name = 'HSI_PCA'

trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(hsi_unet_pca_model, train_loader, val_loader_final, NUM_EPOCHS,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=PATIENCE, model_name=hsi_unet_pca_name,
                                                                            data_source = source)

In [ ]:
trained_model, loss, avg_train_loss_list, avg_val_loss_list = sf_model_training_multiloss(trained_model, train_loader_final, val_loader_final, NUM_EPOCHS_FINAL,
                                                                            ce_loss_fn, dice_loss_fn, optimizer, scaler, scheduler,
                                                                            avg_train_loss_list, avg_val_loss_list,
                                                                            TRAIN_BATCH_SIZE, VAL_BATCH_SIZE,
                                                                            activate_scheduler=False, patience=0, model_name=hsi_unet_pca_name,
                                                                            data_source = source)

In [ ]:
#plot_range = range(NUM_EPOCHS+NUM_EPOCHS_FINAL)

fig, ax = plt.subplots(figsize=(9,6))
ax.plot(range(len(avg_train_loss_list)), avg_train_loss_list, marker='o', linestyle='-', label='Training Loss', color='blue')

# Create a twin Axes sharing the xaxis
ax2 = ax.twinx()
ax2.plot(range(len(avg_val_loss_list)), avg_val_loss_list, marker='o', linestyle='-', label='Validation Loss', color='orange')

# Set labels and title
ax.set_xlabel('Epochs')
ax.set_ylabel('Training Loss', color='blue')
ax2.set_ylabel('Validation Loss', color='orange')
ax.set_title('Training vs Validation Loss')

# Show legend for both axes
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.grid(True)
plt.show()

# Evaluation - HSI GELU Model - Final Model

In [ ]:
test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(trained_model, test_dataset_final, data_source='hsi', visualize = VISUALIZE, mask_shape = (384, 320))

# Evaluation - HSI GELU Model - Final Model

In [ ]:
calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=hsi_unet_pca_name)

# Evaluation - HSI GELU Model - Best Model

In [ ]:
path = '/kaggle/working/'
model_name = f'best_model_{_today}_{hsi_unet_pca_name}.pt'
model_path = os.path.join(path, model_name)
best_model_unet_hsi_pca = load_model(hsi_unet_model_gelu_pca(6), optimizer, scaler, model_path)

test_ds_union, test_ds_intersection, test_ds_numerator, test_ds_denominator, iou_image_pixelwise, dice_image_pixelwise = capture_model_metrics_pixelwise_and_confusion_matrix_sf(best_model_unet_hsi_pca, test_dataset_final, data_source='hsi', visualize=VISUALIZE, mask_shape = (384, 320))

calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True, file_name=hsi_unet_pca_name)


calculate_model_inference_time(best_model_unet_hsi_pca, val_loader_final, 'hsi')

# Post Processing pre Argmax

In [ ]:
test_ds_union = [0,0,0,0,0,0,0,0,0,0]
test_ds_intersection = [0,0,0,0,0,0,0,0,0,0]
test_ds_numerator = [0,0,0,0,0,0,0,0,0,0]
test_ds_denominator = [0,0,0,0,0,0,0,0,0,0]
kernel_size = [1,12,12,1,50,8,6,10,1,4]*2

def probability_based_kernel_post_processing(model, smooth, dataset, kernel_size):

    #create array to capture all ground truth and predictions to calculate final IoU and Dice Score at the end
    ground_truth_all_images=np.zeros((mask_shape[0], mask_shape[1], len(test_dataset_final)))
    prediction_all_images=np.zeros((mask_shape[0], mask_shape[1], len(test_dataset_final)))

    with torch.no_grad():
        model.eval()
        for n, batch in enumerate(dataset):

            #empty numpy mask to fit the one hot encoded classes
            pp_one_hot_pred_masks = np.zeros((384,320, 10))

            rgb_img, hsi_img, mask = batch

            #predict imgs in dataset
            rgb_img = rgb_img.to(DEVICE).unsqueeze(0)
            mask = mask.to(DEVICE)
            softmax = nn.Softmax(dim=1)

            #predicted probabilities
            preds = softmax(model(rgb_img.float())).to('cpu').squeeze(0).permute(1,2,0)
            preds_np = preds.numpy()

            #combined masks for comparison
            preds_argmax = torch.argmax(preds, axis=-1).to('cpu').squeeze(0)

            start = time.time()

            # individual kernels for each defect class
            for i in range(9, 2, -1):
                kernel = np.ones((kernel_size[i],kernel_size[i]),np.uint8)
                radius = int(kernel_size[i]/2)

                #elliptical kernel size
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * radius + 1, 2 * radius + 1))

                #closing operation
                pp_one_hot_pred_masks[:,:,i] = cv2.morphologyEx(preds_np[:,:,i], cv2.MORPH_CLOSE, kernel)

            # individual kernels for each background, fillet front and back class
            for i in range(2, -1, -1):
                kernel = np.ones((kernel_size[i],kernel_size[i]),np.uint8)
                radius = int(kernel_size[i]/2)
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * radius + 1, 2 * radius + 1))

                #dilation operation
                pp_one_hot_pred_masks[:,:,i] = cv2.dilate(preds_np[:,:,i], kernel)

            end = time.time()
            print(f'Time: {end - start}')

            # convert back to torch because evaluation functions only work with torch tensors
            pp_one_hot_pred_masks = torch.from_numpy(pp_one_hot_pred_masks).to('cpu')

            # combine post processed masks
            single_mask = np.argmax(pp_one_hot_pred_masks, axis=-1)

            # add current mask and prediction to stacked array for
            prediction_all_images[:,:,n] = single_mask
            ground_truth_all_images[:,:,n] = mask.to('cpu').numpy()

            #calculate dice and iou score for calculating final IoU and Dice Score at the end
            is_list, u_list=intersection_and_union_all_classes(mask, single_mask, SINGLE_PREDICTION=True)
            n_list, d_list=dice_values_all_classes(mask, single_mask, SINGLE_PREDICTION=True)

            #visualize predictions vs ground truth
            visualize_prediction_vs_ground_truth_overlay_all_sources_postprocessing(rgb_img.squeeze(0), hsi_img.squeeze(0), mask, preds_argmax.squeeze(0), single_mask, 'rgb')

            #print iou and dice score for each individual image
            print('IOU')
            for i in range(len(NUM_UNIQUE_VALUES_LONG)):
                if is_ground_truth_empty(mask)[i] and is_prediction_empty(single_mask)[i]:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: Empty')
                else:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: {is_list[i]/(u_list[i]+smooth):.4f}')
                    #tracking class average of iou across all images
                    test_ds_union[i] += u_list[i]
                    test_ds_intersection[i] += is_list[i]

            print(TXT_COLORS_LONG_COLOR_ONLY[0]+ 'x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x')
            print('Dice')
            for i in range(len(NUM_UNIQUE_VALUES_LONG)):
                if is_ground_truth_empty(mask)[i] and is_prediction_empty(single_mask)[i]:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: Empty')
                else:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: {n_list[i]/(d_list[i]+smooth):.4f}')
                    #tracking class average of iou across all images
                    test_ds_numerator[i] += n_list[i]
                    test_ds_denominator[i] += d_list[i]

                print(TXT_COLORS_LONG_COLOR_ONLY[0])

    #evaluate overall model
    calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True)

probability_based_kernel_post_processing(model, smooth, test_dataset_final, kernel_size)

# Post Processing after argmax

In [ ]:
test_ds_union = [0,0,0,0,0,0,0,0,0,0]
test_ds_intersection = [0,0,0,0,0,0,0,0,0,0]
test_ds_numerator = [0,0,0,0,0,0,0,0,0,0]
test_ds_denominator = [0,0,0,0,0,0,0,0,0,0]
kernel_size = [1,20,20,1,50,8,6,16,1,4]
smooth=1e-8

def region_based_kernel_post_processing(model, smooth, dataset, kernel_size):

    #create array to capture all ground truth and predictions to calculate final IoU and Dice Score at the end
    ground_truth_all_images=np.zeros((mask_shape[0], mask_shape[1], len(dataset)))
    prediction_all_images=np.zeros((mask_shape[0], mask_shape[1], len(dataset)))
    pp_one_hot_pred_masks = np.zeros((384,320, 10),np.uint8)

    with torch.no_grad():
        model.eval()
        for n, batch in enumerate(dataset):
            rgb_img, hsi_img, mask = batch

            rgb_img = rgb_img.to(DEVICE).unsqueeze(0)
            mask = mask.to(DEVICE)
            softmax = nn.Softmax(dim=1)
            preds = torch.argmax(softmax(model(rgb_img.float())),axis=1).to('cpu').squeeze(0)

            #one hot encoding of mask after argmax
            one_hot_pred_masks=F.one_hot(preds.to(torch.int64), num_classes=10).to(DEVICE)

            # individual kernels for each defect class
            for i in range(9, 2, -1):
                kernel = np.ones((kernel_size[i],kernel_size[i]),np.uint8)
                pp_one_hot_pred_masks[:,:,i] = cv2.morphologyEx(one_hot_pred_masks[:,:,i].to('cpu').numpy().astype(np.uint8), cv2.MORPH_OPEN, kernel)

            # individual kernels for each background, fillet front and back class
            for i in range(2, -1, -1):
                kernel = np.ones((kernel_size[i],kernel_size[i]),np.uint8)
                pp_one_hot_pred_masks[:,:,i] = cv2.morphologyEx(one_hot_pred_masks[:,:,i].to('cpu').numpy().astype(np.uint8), cv2.MORPH_CLOSE, kernel)

            #give defect class and fillet front and back more weight than background class
            pp_one_hot_pred_masks[:,:,1:3] = pp_one_hot_pred_masks[:,:,1:3]*2
            pp_one_hot_pred_masks[:,:,3:10] = pp_one_hot_pred_masks[:,:,3:10]*3

            #combine mask
            single_mask_array = np.argmax(pp_one_hot_pred_masks, axis=-1)
            single_mask = torch.from_numpy(single_mask_array)

            # add current mask and prediction to stacked array for confusion matrix
            prediction_all_images[:,:,n] = single_mask_array
            ground_truth_all_images[:,:,n] = mask.to('cpu').numpy()

            #calculate dice and iou score for calculating final IoU and Dice Score at the end
            is_list, u_list=intersection_and_union_all_classes(mask, single_mask, SINGLE_PREDICTION=True)
            n_list, d_list=dice_values_all_classes(mask, single_mask, SINGLE_PREDICTION=True)

            #visualize predictions vs ground truth
            visualize_prediction_vs_ground_truth_overlay_all_sources_postprocessing(rgb_img.squeeze(0), hsi_img.squeeze(0), mask, preds.squeeze(0), single_mask, 'rgb')

            #print iou and dice score for each individual image
            print('IOU')
            for i in range(len(NUM_UNIQUE_VALUES_LONG)):
                if is_ground_truth_empty(mask)[i] and is_prediction_empty(single_mask)[i]:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: Empty')
                else:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: {is_list[i]/(u_list[i]+smooth):.4f}')
                    #tracking class average of iou across all images
                    test_ds_union[i] += u_list[i]
                    test_ds_intersection[i] += is_list[i]

            print(TXT_COLORS_LONG_COLOR_ONLY[0]+ 'x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x')
            print('Dice')
            for i in range(len(NUM_UNIQUE_VALUES_LONG)):
                if is_ground_truth_empty(mask)[i] and is_prediction_empty(single_mask)[i]:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: Empty')
                else:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: {n_list[i]/(d_list[i]+smooth):.4f}')
                    #tracking class average of iou across all images
                    test_ds_numerator[i] += n_list[i]
                    test_ds_denominator[i] += d_list[i]

            print(TXT_COLORS_LONG_COLOR_ONLY[0])

    calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True)

region_based_kernel_post_processing(model, smooth, test_dataset_final, kernel_size)

# CRF-based Post Processing

In [ ]:
#!pip install pydensecrf
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian

test_ds_union = [0,0,0,0,0,0,0,0,0,0]
test_ds_intersection = [0,0,0,0,0,0,0,0,0,0]
test_ds_numerator = [0,0,0,0,0,0,0,0,0,0]
test_ds_denominator = [0,0,0,0,0,0,0,0,0,0]
mask_shape = (384,320)
smooth=1e-8
model = gelu_model

theta_alpha = 20
theta_beta = 15
theta_gamma = 6

def crf_based_post_processing(model, smooth, dataset, mask_shape, theta_a, theta_b, theta_g):

    #create array to capture all ground truth and predictions to calculate final IoU and Dice Score at the end
    ground_truth_all_images=np.zeros((mask_shape[0], mask_shape[1], len(test_dataset_final)))
    prediction_all_images=np.zeros((mask_shape[0], mask_shape[1], len(test_dataset_final)))
    pp_one_hot_pred_masks = np.zeros((384,320, 10),np.uint8)

    #model inference
    with torch.no_grad():
        model.eval()
        for n, batch in enumerate(test_dataset_final):
            rgb_img, hsi_img, mask = batch

            rgb_img = rgb_img.to(DEVICE).unsqueeze(0)
            mask = mask.to(DEVICE)
            softmax = nn.Softmax(dim=1)
            preds = torch.argmax(softmax(model(rgb_img.float())),axis=1).to('cpu').squeeze(0)

            #convert original img and annotated img into numpy arrays
            original_image=rgb_img.to('cpu').squeeze().permute(1,2,0).numpy()
            annotated_image=preds.to('cpu').numpy().astype(np.uint32)

            #from here: code snippets from git@github.com:lucasb-eyer/pydensecrf.git
            #and from here: code snippets from git@github.com:dhawan98/Post-Processing-of-Image-Segmentation-using-CRF.git
            #number of classes in dataset
            n_labels_a = 10

            #flatten segmentation mask
            labels_a = annotated_image.flatten()

            #Setting up the CRF model
            d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels_a)

            # get unary potentials (neg log probability)
            U = unary_from_labels(labels_a, n_labels_a, gt_prob=0.90, zero_unsure=False)

            #calculate Gibbs energy
            d.setUnaryEnergy(U)

            # This adds the color-independent term, features are the locations only.
            # smoothing kernel
            d.addPairwiseGaussian(sxy=(theta_gamma, theta_gamma), compat=3, kernel=dcrf.DIAG_KERNEL,
                              normalization=dcrf.NORMALIZE_SYMMETRIC)

            # This adds the color-dependent term, i.e. features are (x,y,r,g,b).
            # appearance kernel
            d.addPairwiseBilateral(sxy=(theta_alpha, theta_alpha), srgb=(theta_beta, theta_beta, theta_beta), rgbim=original_image.astype(np.uint8),
                               compat=10,
                               kernel=dcrf.DIAG_KERNEL,
                               normalization=dcrf.NORMALIZE_SYMMETRIC)

            #Run CRF model inference for x steps
            Q = d.inference(1)

            # Find out the most probable class for each pixel.
            MAP = np.argmax(Q, axis=0)

            # Convert the MAP (labels) back to the corresponding colors and save the image.
            post_processed_mask=MAP.reshape(annotated_image.shape)

            ####
            #code snippets from github repos end here

            #convert mask back to torch tensor for evaluating purposes
            single_mask = torch.from_numpy(post_processed_mask)

            #calculate dice and iou score for calculating final IoU and Dice Score at the end
            is_list, u_list=intersection_and_union_all_classes(mask, single_mask, SINGLE_PREDICTION=True)
            n_list, d_list=dice_values_all_classes(mask, single_mask, SINGLE_PREDICTION=True)

            #visualize predictions vs ground truth
            visualize_prediction_vs_ground_truth_overlay_all_sources_postprocessing(rgb_img.squeeze(0), hsi_img.squeeze(0), mask, preds.squeeze(0), single_mask, 'rgb')

            #print iou and dice score for each individual image
            print('IOU')
            for i in range(len(NUM_UNIQUE_VALUES_LONG)):
                if is_ground_truth_empty(mask)[i] and is_prediction_empty(single_mask)[i]:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: Empty')
                else:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: {is_list[i]/(u_list[i]+smooth):.4f}')
                    #tracking class average of iou across all images
                    test_ds_union[i] += u_list[i]
                    test_ds_intersection[i] += is_list[i]

            print(TXT_COLORS_LONG_COLOR_ONLY[0]+ 'x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x')
            print('Dice')
            for i in range(len(NUM_UNIQUE_VALUES_LONG)):
                if is_ground_truth_empty(mask)[i] and is_prediction_empty(single_mask)[i]:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: Empty')
                else:
                    print(f'{TXT_COLORS_LONG_COLOR_ONLY[i]} - {CLASSES_LONG[i]}: {n_list[i]/(d_list[i]+smooth):.4f}')
                    #tracking class average of iou across all images
                    test_ds_numerator[i] += n_list[i]
                    test_ds_denominator[i] += d_list[i]

            print(TXT_COLORS_LONG_COLOR_ONLY[0])

        calculate_model_metrics(test_ds_intersection, test_ds_union, test_ds_numerator, test_ds_denominator, defects_only=True)

crf_based_post_processing(model, smooth, test_dataset_final, mask_shape, theta_alpha, theta_beta, theta_gamma)